<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
# importar el módulo piplite
import piplite
#  instalar la biblioteca base de folium.
await piplite.install(['folium'])
# instalar el paquete pandas, para el análisis de datos.
await piplite.install(['pandas'])

In [2]:
# Usar import e importar la biblioteca pandas, asignarle el alias pd.
import pandas as pd
# importar la biblioteca base de folium.
import folium

In [3]:
# importar el plugin MarkerCluster de folium
from folium.plugins import MarkerCluster
# importar el plugin MousePosition de folium 
from folium.plugins import MousePosition
# importar el plugin DivIcon de folium 
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
# Importar la función fetch para obtener recursos dee la web. 
from js import fetch
# Importar  el módulo io para manejar operaciones de entrada y salida en archivos. 
import io
# Definir la URL del Archivo CSV `spacex_launch_geo.csv`:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
# Usar fetch para Descargar el Archivo:
resp = await fetch(URL)
# Convertir la Respuesta io a un Formato Legible por pandas:
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
# Leer el Archivo CSV con pandas:
spacex_df = pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]    # Seleccionar del fichero spacex_csv las subcolumnas `Launch Site`, `Lat`, `Long`y `class`
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()    # Agrupar por columna 'Launch Site' y tomar la primera fila de cada grupo
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]    # Seleccionar únicamente las columnas 'Launch Site', 'Lat' y 'Long' del DataFrame resultante
launch_sites_df    # Mostrar el DataFrame final

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Crear el folium map
nasa_coordinate = [29.559684888503615, -95.0830971930759]   # Ubicación inicial: coord.NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)   # zoom 10

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
# Crear un folium circle en coordenadas de NASA JSC con un mensaje emergente mostrando el nombre
circle = folium.Circle(
    nasa_coordinate,   
    radius=1000,    
    color='#d35400',    
    fill=True    
).add_child(folium.Popup('NASA Johnson Space Center'))   

# Crear con un folium marker un marcador estándar en un punto específico mostrando el nombre.
marker = folium.map.Marker(
    nasa_coordinate, 
    # Crear icono, como etiqueta de texto (permite HTML para personalizar la apariencia del marcador)
    icon=DivIcon( 
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC', 
        )
    )

site_map.add_child(circle)   # Añadir el círculo al mapa
site_map.add_child(marker)   # Añadir el marcador al mapa

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Inicializar el mapa con zoom 5
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# En cada columna del dataframe launch_sites, iterar extrayendo sus coordenadas.
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]

    # Añadir círculos naranja #d35400 con radio de 1000 metros alrededor de cada sitio
    folium.Circle(
        coordinate,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)  # Añadir etiqueta (popup) con el nombre de cada Launch Site que se mostrará dentro del círculo, al hacer clic en él.
    
    # Añadir marcador en cada Launch Site con un icono personalizado, mediante un div HTML (se muestra directamente sin necesidad de clics).
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )  # El DivIcon muestra el nombre del sitio en color naranja y con fuente tamaño 12.
    ).add_to(site_map)

# Mostrar el mapa actualizado
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Exploinge the map by zoom-in/out the marked areas I could observe:

- Are all launch sites in proximity to the Equator line?
- None of these sites are extremely close to the equator, which is located at latitude 0°. However, the sites at the Kennedy Space Center (KSC) and Cape Canaveral Space Force Station (CCAFS) are closer to the equator if we compared this to the site at Vandenberg (VAFB)..

- Are all launch sites in very close proximity to the coast?
- All the  sites are extremely close to the coast. This proximity to the coast helps both logistically and strategically for safety, as launching rockets from coastal locations allows the rocket's first stages to safely fall into the ocean, away from populated areas, avoiding them from falling in inhabited areas

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
# Mostrar las ultimas 10 filas del DataFrame spacex_df
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
# Crear un objeto MarkerCluster
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [12]:
# Asignar a los markers de launch_outcome, color en función del valor de 'class'
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# aplicando assign_marker_color a cada valor de Class, crear nueva columna 'marker_color'
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Mostrar las ultimas 10 filas del DataFrame spacex_df 
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Añadir marker_cluster al site_map actual
site_map.add_child(marker_cluster)

# Iterar en cada fila de spacex_df
for index, record in spacex_df.iterrows():
    # Obtener las coordenadas para crear el objeto Marker
    coordinate = [record['Lat'], record['Long']]
    
    # Crear el marcador de folium al que se añaden sus propiedades 
    marker = folium.Marker(
        location=coordinate,  # Ubicación del marcador
        icon=folium.Icon(color='white', icon_color=record['marker_color']),  # Icono del marcador
        popup=record['Launch Site']  # Información emergente
    )
    
    # Añadir el nuevo marcador Marker al clúster de marcadores marker_cluster
    marker_cluster.add_child(marker)

# Devolver el mapa actualizado
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

- I perceived:
  
- KSC LC-39A has 13, of which only 3 are failures and 10 are successful.
- CCAFS SLC-40 has 7, of which 4 are failures and only 3 are successful.
- CCAFS LC-40 has 26, of which 19 are failures and only 7 are successful.
- VAFB SLC-4E has 10, of which 6 are failures and only 4 are successful.

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# 1. Formateador: 
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
# 2. crear objeto MousePosition
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',  # Prefijo a mostrar
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)  # Añadir el objeto mouse_position al site_map

# 3. mostrar el mapa actualizado.
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [15]:
# Función de cálculo
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [17]:
# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.56342
launch_site_lon = -80.57678
# Coordenadas del punto de la costa
coastline_lat = 28.56345
coastline_lon = -80.56796

# Calcular la distancia
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f'Distance to coastline: {distance_coastline:.2f} Km')

Distance to coastline: 0.86 Km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [18]:
# Crear y añadir un folium.Marker de marcador de distancia en el punto de costa en el punto de costa seleccionado
distance_marker = folium.Marker(
    # Coordenadas donde se pondrá el marcador de la costa mostrando la distancia entre el punto de la costa y el sitio de lanzamiento
    [coastline_lat, coastline_lon],
    # usar divIcon para personalizar el icono del marcador
    icon=DivIcon( 
        icon_size=(20,20),  # tamaño del icono
        icon_anchor=(0,0),  # Posición: usar posición del icono folium.Marker en el mapa
        # texto del marcador (en este caso 'distance')  y formato, como kilómetros, tamaño 12 y color naranja) 
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

site_map.add_child(distance_marker)  # Añadir el marcador al mapa
site_map  # Mostrar el mapa actualizado con el marcador de distancia

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
# Crear un PolyLine desde el launch site a la costa
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)

site_map.add_child(lines)  # Verificar el mapa

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [20]:
# Crear una PolyLine desde el launch site al ferrocarril
railway_lat = 28.5721   
railway_lon = -80.5853  
railway_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]], 
    weight=2,
    color='purple'
)

# Añadir la PolyLine al mapa
site_map.add_child(railway_line)

# Visualizar el mapa actualizado
site_map

In [21]:
# Crear una PolyLine desde el launch site a la carretera
road_lat = 28.55412   
road_lon = -80.58883  
road_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [road_lat, road_lon]], 
    weight=2,
    color='green'
)

# Añadir la PolyLine al mapa
site_map.add_child(road_line)

# Visualizar el mapa actualizado
site_map

In [22]:
# Crear una PolyLine desde el launch site a la ciudad
city_lat = 28.4158   
city_lon = -81.2989  
city_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]], 
    weight=2,
    color='red'
)

# Añadir la PolyLine al mapa
site_map.add_child(city_line)

# Visualizar el mapa actualizado
site_map

After plotting distance lines to the proximities, I can answer:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

- Yes, is close to railways and roads, good for transport of goods and people.
- It´s also close to coastline, as comented before this is good for logistics.
- It's situation, close to coastline and away from populated residential areas, it's perfect for safety, (as comented, allows the rocket's first stages to safely fall into the ocean and cause no harm).

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
